# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Сформулируем цель исследования**

Цель - разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок на основе статистики о платёжеспособности клиентов.


**Объект исследования**

Объектом исследования выступает статистика о платёжеспособности клиентов, содержащаяся в файле `/datasets/data.csv`

**Сформулируем задачи, которые предполагается решить для достижения цели исследования**

1. Определить, есть ли зависимость между семейным положением и фактом погашения кредита в срок.
2. Определить, есть ли зависимость между количеством детей и фактом погашения кредита в срок.
3. Определить, есть ли зависимость между уровнем дохода и фактом погашения кредита в срок.
4. Определить, как разные цели кредита влияют на его возврат в срок.


**План исследования**
1. [Обзор данных](#section_1)
2. [Предобработка данных](#section_2)
3. [Решение задач исследования](#section_3)
4. [Выводы](#section_4)
5. [Заполнение чек-листа](#section_5)

<a id='section_1'></a>
## Шаг 1. Обзор данных


Импортируем `pandas`

In [1]:
import pandas as pd

Прочитаем файл `/datasets/data.csv` 

In [2]:
df=pd.read_csv('/datasets/data.csv')

Выведем первые 10 строк на экран для ознакомления

In [3]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Согласно исходным данным, в стобцах содержится следующая информация:

• `children` — количество детей в семье

• `days_employed` — общий трудовой стаж в днях

• `dob_years` — возраст клиента в годах

• `education` — уровень образования клиента

• `education_id` — идентификатор уровня образования

• `family_status` — семейное положение

• `family_status_id` — идентификатор семейного положения

• `gender` — пол клиента

• `income_type` — тип занятости

• `debt` — имел ли задолженность по возврату кредитов

• `total_income` — ежемесячный доход

• `purpose` — цель получения кредита

Любопытно, что в общем трудовом стаже встречаются отрицательные значения и очень большие, и с дробной частью. Возможно ошибка при заполнении таблицы, конвертации в число. Можно уточнить, но нам этот столбец не критичен, поэтому можем оставить как есть.

В столбце с образованием встречаются разные варианты записей. Можем поправить, хоть нам это и не критично.

В каких единицах уровень дохода - не указано. Предполагаем, что в рублях. Но пока нет подтверждения, будем считать некими условными денежными единицами (у.д.е.)

Посмотрим общую информацию о таблице:

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Выводы**:

Файл прочитался без ошибок и столбцы соответствуют описанию. Названия столбцов корректны (без пробелов, латиницей и нет заглавных букв). Важная информация о количестве детей, семейном положении присутствует, уровне доходов, целях кредита и наличии задолженности присутствует. Однако количество значений в столбцах отличается, что говорит о наличии пропусков в столбцах. В столбце со стажем, встречаются некорректые значения. В столбце с образованием встречаются разные варианты записей. С этим разберёмся на этапе предобработки данных. 

<a id='section_2'></a>
## Шаг 2. Предобработка данных

### Обработка пропусков

Как уже определили выше, в столбцах есть явные пропуски, которые попробуем обработать

In [5]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Явные пропуски встречаются в столбцах с общим трудовым стажем в днях и ежемесячным доходом. По большому счёту для ответа на поставленные в исследовании вопросы информация о стаже не важна, однако ежемесечный доход важен и нам его желательно корректно заполнить. Возможно причина пропусков в некорректном заполнении формы. Маловероятно, что банк выдал кредит людям без дохода.

Инерес вызывает факт, что количество пропусков этих столбцах совпадает. Проверим предположение, что пропуски встречаются в одних записях. Проверим это. Определим количество строк, у которых пропуски и в `days_employed` и в `total_income`

In [6]:
len(df.loc[(df['days_employed'].isna()==True) & (df['total_income'].isna()==True)])

2174

Цифры совпадают, что говорит о том, что наше предположение верно и пропуски одновременно в двух стобцах. Логично предположить, что такие пропуски имеют безработные. Проверим, кокой тип занятости `income_type` имеют записи с пропусками.

In [7]:
df.loc[(df['days_employed'].isna()==True) & (df['total_income'].isna()==True)]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Безработных нет)) Тип занятости - различный. Посмотрим, сколько вообще каких бывает типов занятости в исходной таблице

In [8]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Из общего перечня типов занятости в перечень с пропусками не попали студент, безработный и в декрете. Для дальнейшей обработки записей можем их отбросить, либо чем-то заменить. Если отбросить - то потеряем примерно 10% каждой категории и 50% для предпринимателя. Если не отбросим, то возможны искажения реальных цифр. Эти искажения могут возникнуть только для 3-ей задачи. 

Логичным видится следующее, пропуски в стобце `days_employed`, как несущественные, заменим 0. 

In [9]:
df.loc[df['days_employed'].isna()==True, 'days_employed'] = 0

Проверим, что NaN ушли из столбца

In [10]:
df.isna().sum()

children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Верно.Для `total_income` заменим значение средним заработком в каждом типе занятости и добавим столбец `not_trusted` с признаком, что значение взято по среднему. Этот столбец учтём, когда будем считать зависимость от заработка. Значение True столбец принимает, если значение `total_income` взято по среднему.

In [11]:
df['not_trusted'] = df['total_income'].isna()

Проверим, совпадает ли сумма `not_trusted` с количеством пропусков в столбце `total_income`.

In [12]:
df['not_trusted'].sum()

2174

Вышло. Теперь заменим пропуски средним. Для начала определим средний зарабаток в каждой категории.

In [13]:
mean_incomes=df.groupby('income_type')['total_income'].mean()

In [14]:
mean_incomes

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        170898.309923
компаньон          202417.461462
пенсионер          137127.465690
предприниматель    499163.144947
сотрудник          161380.260488
студент             98201.625314
Name: total_income, dtype: float64

Определим уникальные значения `income_type` в таблице, имеющие пропуски

In [15]:
incomes=df[df['total_income'].isna()==True]['income_type'].unique()

In [16]:
incomes

array(['пенсионер', 'госслужащий', 'компаньон', 'сотрудник',
       'предприниматель'], dtype=object)

Произведём замену

In [17]:
for income in incomes:
    df.loc[(df['total_income'].isna()==True) & (df['income_type']==income), 'total_income']=mean_incomes[income]

Проверим, изменились ли средние заработки

In [18]:
df.groupby('income_type')['total_income'].mean()

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        170898.309923
компаньон          202417.461462
пенсионер          137127.465690
предприниматель    499163.144947
сотрудник          161380.260488
студент             98201.625314
Name: total_income, dtype: float64

Средние заработки не изменились. Проверим, остались ли пропуски в таблице

In [19]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
not_trusted         0
dtype: int64

Пропуски ушли. Попробуем разобраться с другими значимыми для нас столбцами. Для начала помотрим на `debt`

In [20]:
df['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

Тут всё нормально. Посмотрим на семейное положение `family_status`

In [21]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Тоже нас устроит. Посмотрим на уровни дохода `total_income`.

In [22]:
df['total_income'].describe()

count    2.152500e+04
mean     1.673959e+05
std      9.790695e+04
min      2.066726e+04
25%      1.077982e+05
50%      1.519313e+05
75%      2.024175e+05
max      2.265604e+06
Name: total_income, dtype: float64

Тоже более-меннее. Посмотрим количество уникальных значений в столбце `children`.

In [23]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Любопытно, что встречается -1 ребёнок и 20. Здесь явная ошибка. Посмотрим на эти записи

In [24]:
df[df['children'] == -1].sort_values(by='dob_years').head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,not_trusted
14357,-1,-268.337037,23,среднее,1,в разводе,3,F,компаньон,0,92257.579312,покупка жилой недвижимости,False
3814,-1,-3045.290443,26,Среднее,1,гражданский брак,1,F,госслужащий,0,131892.785435,на проведение свадьбы,False
11617,-1,-370.827130,27,среднее,1,женат / замужем,0,F,компаньон,0,164591.260338,покупка жилья для сдачи,False
9574,-1,-2710.419901,28,высшее,0,женат / замужем,0,M,госслужащий,0,303137.161001,получение дополнительного образования,False
19417,-1,350340.760224,28,среднее,1,в разводе,3,F,пенсионер,0,52872.993654,автомобили,False


Видим, что -1 встречается у самого молодого человека в 23 года. Вероятно знак - возник, когда в графе дети поставили прочерк(-). Так как таких значений не много, то можем заменить это значение на 0, и на общую статистуку вляние будет не велико.

In [25]:
df.loc[df['children'] == -1, 'children'] = 0

Теперь посмотрим на записи, у которых 20 детей.

In [26]:
df[df['children'] == 20].sort_values(by='dob_years').head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,not_trusted
2941,20,-2161.591519,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля,False
606,20,-880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья,False
3671,20,-913.161503,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,101255.492076,на покупку подержанного автомобиля,False
5315,20,-2047.754733,24,среднее,1,женат / замужем,0,F,сотрудник,0,100415.236833,покупка коммерческой недвижимости,False
12909,20,0.000000,25,среднее,1,женат / замужем,0,M,сотрудник,0,161380.260488,операции с жильем,True


Выяснили, что есть нули в графе возраст. По-хорошему их надо обработать, но для нас это не значимая графа и оставим как есть. Но к 21 году иметь 20 детей - думаю, такое невозможно. В наших реалиях, логично предположить что из-за конвертации форматов добавился 0. В любом случае, выборка снова не большая. Если заменим на 2, то существенного влияния на результат это иметь не будет.

In [27]:
df.loc[df['children'] == 20, 'children'] = 2

Посмотрим результат

In [28]:
df['children'].value_counts()

0    14196
1     4818
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Остальные значения более реальны. Оставляем так.

По большому счёту, надо разобраться с не существенными стобцами. Однако это не входит в задачи исследования. Возможно впоследствии появится информация, которая поможет корректно разобраться и с не существенными столбцами.

**Вывод**

В ходе предобработки выяснилось, что есть пропуски одновременно в значимом столбце со среднемесечным доходом `total_income` и общим количеством трудового стажа `days_employed`. Возможная причина - пропуски в заполнении формы. Вместо пропусков в столбец со стажем записали 0. Столбец не критичный и этим мы просто пометили, что значений нет. В столбец со среднемесечным доходом для пропущенных значений записали средний доход по категории занятости `income_type` к которой пренадлажат записи. Дополнительно добавили столбец `not_trusted` имеющий значение True, если для записи взят средний заработок. Проверили данные о семейном положении, наличии задоженности. Исправили нестыковки в количестве детей. Втаком виде можем искать зависимости.

### Замена типа данных

Для сокращения числовых записей в стаже логично заменить тип данных *float* на *int*. Используем `astype()`, как самое очевидное решение.

In [29]:
df['days_employed']=df['days_employed'].astype(int)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
not_trusted         21525 non-null bool
dtypes: bool(1), float64(1), int64(6), object(5)
memory usage: 2.0+ MB


In [31]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,not_trusted
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,False
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,False
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,False
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,False
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,False


**Вывод**

Мы привели значения в `days_employed` к типу *int*. Тем самым сократили запись и сделали её удобной для чтения. Значения в `total_income` не приводили, так как наверняка не известно, какие знаки после запятой могут быть значимы.

### Обработка дубликатов

Найдём явные дубликаты записей. По условию не указано, возможна ли ситуация, когда записи - не дубликаты, а просто разные люди, у которых всё совпало. Считаем, что такое невозможно.

In [32]:
df.duplicated().sum()

54

In [33]:
df=df.drop_duplicates().reset_index(drop=True)

Проверим, что удалили дубликаты

In [34]:
df.duplicated().sum()

0

Если были явные дубликаты, то возможны и неявные, например, когда в столбце `education` запись будет записана по разному. Можно конечно удалить столбец и орентироваться по `education_id`, но постараемся оставить столбец.
Проверим, каким образом записаны сведения об образовании

In [35]:
df['education'].value_counts()

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Встречаются записи в различных видах. Приведём всё к строчным буквам.

In [36]:
df['education'] = df['education'].str.lower()

Проверим, как получилось.

In [37]:
df['education'].value_counts()

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Да, всё вышло. 

Проверим, появились ли дубликаты.

In [38]:
df.duplicated().sum()

17

Да, появились. Удаляем их и перезагружаем индекс

In [39]:
df=df.drop_duplicates().reset_index(drop=True)

Проверяем, что дубликатов не осталось

In [40]:
df.duplicated().sum()

0

**Вывод**

Таким образом в талице были обнаружены явные и неявные дубликаты. Неявные возникли из-за различиных записей в столбце `education`. Все дубликаты были удалены.

### Лемматизация

Определим количество уникальных значений в столбце с целью кретита

In [41]:
len(df['purpose'].unique())

38

Не так и много. Посмотрим их

In [42]:
purposes=df['purpose'].unique()

In [43]:
purposes

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Есть логические категории целей. Их не много. Проведём лемматизацию. Для этого список лематизируем список уникальных целей.

In [44]:
from pymystem3 import Mystem
m = Mystem() 
lemmas=[]
for purpose in purposes:
    lem = m.lemmatize(purpose)
    if lem not in lemmas:
        lemmas.append(lem)          
print(lemmas)

[['покупка', ' ', 'жилье', '\n'], ['приобретение', ' ', 'автомобиль', '\n'], ['дополнительный', ' ', 'образование', '\n'], ['сыграть', ' ', 'свадьба', '\n'], ['операция', ' ', 'с', ' ', 'жилье', '\n'], ['образование', '\n'], ['на', ' ', 'проведение', ' ', 'свадьба', '\n'], ['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n'], ['покупка', ' ', 'недвижимость', '\n'], ['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n'], ['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n'], ['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n'], ['недвижимость', '\n'], ['строительство', ' ', 'недвижимость', '\n'], ['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n'], ['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n'], ['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n'], ['строительство', ' ', 'жилой', ' ', 'недвижимость', '\n'], ['жилье', '\n'], ['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n'], ['автомобиль', '\n'], ['заниматьс

На основании данных, можно выделить ледующие категории:

\'недвижимость\' - сюда входят приобретение жилья и операции с недвижимостью,

\'автомобиль\'   - сюда входят сделки с автомобилем,

\'образование\'  - сюда входят кретиты на получение образования,

\'свадьба\'      - сюда входят кредиты на организаццию свадьбы


**Вывод**

Мы провели лемматизацию целей кредита. Выявили, что можно выделить 4 категории целей:'недвижимость','автомобиль', 'образование','свадьба'. На основании этого проведём категоризацию целей кредита.

### Категоризация данных

Категоризируем по целям кредита. Напишем функцию `purpose_categorize()`, которая по тексту будет возвращать категорию из перечисленных.

In [45]:
def purpose_categorize(purpose):
    words = m.lemmatize(purpose)
    if 'образование' in words:
        return 'образование'
    if 'автомобиль' in words:
        return 'автомобиль'
    if 'свадьба' in words:
        return 'свадьба'
    if 'недвижимость' in words or 'жилье' in words:
        return 'недвижимость'
    else:
        return 'без категории'   

Добавим ещё один столбец с категорией целей кредита purpose_category и заполним его.

In [46]:
df['purpose_category'] = df['purpose'].apply(purpose_categorize)

Посмотрим на первые строки таблицы и убедимся, что всё прошло успешно.

In [47]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,not_trusted,purpose_category
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,False,недвижимость
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,False,автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,False,недвижимость
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,False,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,False,свадьба


Посмотрим, сколько значений в каждой категории.

In [48]:
df['purpose_category'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_category, dtype: int64

Категоризируем по уровню доходов. Создадим столбец `income_category` и разобъем на 5 уровней доходов

In [49]:
df['income_category']=pd.qcut(df['total_income'],5)

Посмотрим, на какие категории разбилась таблица и сколько значений в каждой

In [50]:
df['income_category'].value_counts()

(214618.469, 2265604.029]    4291
(162639.672, 214618.469]     4291
(98538.268, 135448.912]      4291
(20667.263, 98538.268]       4291
(135448.912, 162639.672]     4290
Name: income_category, dtype: int64

Категоризация по уровню доходов произведена

**Вывод**

Мы провели категоризацию записей таблицы по целям кредита, использую данные предыдущего шага. В запроса преобладают сделки с недвижимостью. На 2 месте - сделки с автомобилем, на 3- с небольшим отставанием - расходы на образование. На 4-свадьба.
По уровню доходов мы разбили на 5 категорий, количество записей в которых примерно одинаково.

<a id='section_3'></a>
## Шаг 3. Ответы на задачи исследования

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [51]:
df.groupby('children')['debt'].agg(['count', 'mean'])

,count,mean
children,,
0,14138,0.075258
1,4808,0.092346
2,2128,0.094925
3,330,0.081818
4,41,0.097561
5,9,0.000000


**Вывод**

Видим, что с люди с количеством детей 5 не имеет задолженности. Но таких очень мало записей. Не известно, насколько показательна выборка. Если отбросить людей с 5-ю детьми, то наименьшую задолженность имеют люди без детей, а наибольшую - те у которых 4 ребёнка. Звучит логично.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [52]:
df.groupby('family_status')['debt'].agg(['count','mean']).sort_values(by='mean')

,count,mean
family_status,,
вдовец / вдова,959,0.065693
в разводе,1195,0.071130
женат / замужем,12339,0.075452
гражданский брак,4151,0.093471
Не женат / не замужем,2810,0.097509


**Вывод**

Видим, что наименьшую задолженность имеют люди, имеющее записи вдовец/вдова. А наибольшую - те кто не женат/не замужем.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [53]:
df.groupby('income_category')['debt'].agg(['count', 'mean'])

,count,mean
income_category,,
"(20667.263, 98538.268]",4291,0.080168
"(98538.268, 135448.912]",4291,0.082731
"(135448.912, 162639.672]",4290,0.089510
"(162639.672, 214618.469]",4291,0.083430
"(214618.469, 2265604.029]",4291,0.069914


Логично, что люди с высоким уровнем дохода имеют меньшую задолженность по кредиту. В тоже время наибольшую задолженность имеют люди не с низким, а со средним уровнем доходов.

Проверим, изменится ли картина, если будем учитывать только оригинальные данные, т.е. те в которых пропуски не заполнялись.

In [54]:
df[df['not_trusted']==False].groupby('income_category')['debt'].agg(['count', 'mean'])

,count,mean
income_category,,
"(20667.263, 98538.268]",4291,0.080168
"(98538.268, 135448.912]",4291,0.082731
"(135448.912, 162639.672]",2834,0.088567
"(162639.672, 214618.469]",3645,0.088066
"(214618.469, 2265604.029]",4290,0.069930


Видим, что ситуация практически не изменилась. Поэтому можно утверждать, что замена среднемесечного дохода средним значением была оправдана.

**Вывод**

Выяснили,что люди с высоким уровнем дохода имеют меньшую задолженность по кредиту. В тоже время наибольшую задолженность имеют люди не с низким, а со средним уровнем доходов.

- Как разные цели кредита влияют на его возврат в срок?

In [55]:
df.groupby('purpose_category')['debt'].agg(['count', 'mean']).sort_values(by='mean')

,count,mean
purpose_category,,
недвижимость,10811,0.072334
свадьба,2324,0.080034
образование,4013,0.092200
автомобиль,4306,0.093590


**Вывод**

Видим, что наибольее вероятная задолженности у категории кредитов на автомобиль, а наименьшая - на операции с недвижимостью. И на недвижимость берут больше кредитов.

<a id='section_4'></a>
## Шаг 4. Общий вывод

В ходе исследования изучили представленный файл `/datasets/data.csv`. Заполнили пропуски в значимом столбце со средним заработком. Исправили подозрительные значения в количестве детей. Провели категоризацию записей по уровню среднего заработка и целей получения кредита.

Установили:
1. Зависимость между возвратом кредита в срок и семейным положением имеется. Наименьшую задолженность имеют люди с семейным положением вдовец/вдова. Наибольшую - не женат/не замужем.
2. Зависимость между возвратом кредита в срок и количеством детей имеется. Наименьшую задолженность имеют люди с одним ребёнком. Наибольшую - люди с 4-мя. Люди с 5-ю детьми имеют малое к-во значений, чтобы их учитывать.
3. Зависимость между возвратом кредита в срок и уровнем дохода имеется. Люди с высоким уровнем дохода имеют меньше задолженностей. Наибольшую задолженность имеют люди со средним заработком. 
4. Зависимость между возвратом кредита в срок и целью кредита имеется. Наименьшую задолженность имеют кредиты, выданные на операции снедвижимостью. Наибольшую - на автомобиль.